<a href="https://colab.research.google.com/github/niloRoch/Chromext--Brightwords-project/blob/main/FIX_random_forest_83_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


from google import colab as cl


In [2]:
#importando arq
file_upload = cl.files.upload()

Saving Lotof.csv to Lotof.csv


In [20]:
# Carregar os dados
file_path = "Lotof.csv"
df = pd.read_csv(file_path, sep=';')

In [21]:
# Funções para criar features adicionais
def contar_pares(lista):
    return sum(1 for num in lista if num % 2 == 0)

def contar_primos(lista):
    primos = {2, 3, 5, 7, 11, 13, 17, 19, 23}
    return sum(1 for num in lista if num in primos)

def contar_fibonacci(lista):
    fibonacci = {1, 2, 3, 5, 8, 13, 21}
    return sum(1 for num in lista if num in fibonacci)

def contar_multiplos_de_3(lista):
    return sum(1 for num in lista if num % 3 == 0)

def soma_dezenas(lista):
    return sum(lista)

def contar_sequencias(lista):
    count = 0
    lista = sorted(lista)
    for i in range(len(lista) - 1):
        if lista[i] + 1 == lista[i + 1]:
            count += 1
    return count

# Preparar os dados
dezenas = df.iloc[:, 2:17].values  # As colunas Bola1 a Bola15 contêm os números sorteados

# Calcular a frequência de cada número (1 a 25) em todos os sorteios
frequencia_numeros = np.zeros((dezenas.shape[0], 25))
for i in range(dezenas.shape[0]):
    for j in range(dezenas.shape[1]):
        numero = dezenas[i, j] - 1  # Subtrair 1 para indexar corretamente (números de 1 a 25)
        frequencia_numeros[i, numero] += 1

# Criar rótulos (y)
y = np.zeros((dezenas.shape[0], 25))
for i in range(dezenas.shape[0]):
    for j in range(dezenas.shape[1]):
        numero = dezenas[i, j] - 1
        y[i, numero] = 1

# Criar DataFrame para facilitar a manipulação
historico = pd.DataFrame(dezenas)


In [22]:
# Adicionar features adicionais
X = pd.DataFrame(frequencia_numeros)
X['pares'] = historico.apply(lambda row: contar_pares(row.values), axis=1)
X['impares'] = 15 - X['pares']
X['primos'] = historico.apply(lambda row: contar_primos(row.values), axis=1)
X['fibonacci'] = historico.apply(lambda row: contar_fibonacci(row.values), axis=1)
X['multiplos_de_3'] = historico.apply(lambda row: contar_multiplos_de_3(row.values), axis=1)
X['soma_dezenas'] = historico.apply(lambda row: soma_dezenas(row.values), axis=1)
X['sequencias'] = historico.apply(lambda row: contar_sequencias(row.values), axis=1)
X['numeros_repetidos'] = (historico.shift(1) == historico).sum(axis=1)

In [23]:
# Configurar TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)  # 5 folds temporais

In [24]:
# Converter todos os nomes das colunas para strings
X.columns = X.columns.astype(str)

In [25]:
# Criar o modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)

In [26]:
# Avaliar o modelo com TimeSeriesSplit
rf_scores = []
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [27]:
# Treinar o modelo
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)

In [28]:
# Fazer previsões
y_pred = rf_model.predict(X_test)

# Calcular acurácia
acuracia = accuracy_score(y_test, y_pred)
rf_scores.append(acuracia)

# Média da acurácia
print(f"Acurácia média do Random Forest com TimeSeriesSplit: {np.mean(rf_scores):.2f}")

Acurácia média do Random Forest com TimeSeriesSplit: 0.83


In [29]:
# Prever as probabilidades para todos os números (1 a 25)
X_pred = np.zeros((25, X.shape[1]))  # Matriz de características para previsão
for i in range(25):
    X_pred[i, i] = 1  # Frequência de cada número

In [30]:
# Adicionar features adicionais para previsão
X_pred = pd.DataFrame(X_pred, columns=X.columns)
X_pred['pares'] = [contar_pares([i + 1]) for i in range(25)]
X_pred['impares'] = 1 - X_pred['pares']
X_pred['primos'] = [contar_primos([i + 1]) for i in range(25)]
X_pred['fibonacci'] = [contar_fibonacci([i + 1]) for i in range(25)]
X_pred['multiplos_de_3'] = [contar_multiplos_de_3([i + 1]) for i in range(25)]
X_pred['soma_dezenas'] = [soma_dezenas([i + 1]) for i in range(25)]
X_pred['sequencias'] = [contar_sequencias([i + 1]) for i in range(25)]
X_pred['numeros_repetidos'] = 0  # Não há repetição para previsão de um único número

In [31]:
# Prever as probabilidades
probabilidades = rf_model.predict_proba(X_pred)  # Probabilidade de cada número ser sorteado


In [32]:
# Calcular a média das probabilidades para cada número
probabilidades_medias = np.mean([prob[:, 1] for prob in probabilidades], axis=0)

In [33]:
# Selecionar os 18 números com maior probabilidade
top_18 = np.argsort(probabilidades_medias)[-18:] + 1  # +1 para ajustar os índices (números de 1 a 25)

In [34]:
# Ordenar as dezenas em ordem crescente
top_18_ordenado = sorted(top_18)

In [35]:
# Exibir os 18 números mais prováveis em ordem crescente
print("18 dezenas com maior probabilidade de serem sorteadas (em ordem crescente):")
print(top_18_ordenado)

18 dezenas com maior probabilidade de serem sorteadas (em ordem crescente):
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 25]
